# ham_or_spam    

Julia example of Naive Bayes spam mail classification.   

## Bayes Theorem

Albeit being arguably the simplest algorithm in machine learning, [Bayes Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) is very powerful and practical in applications such as spam mail classification, recommendation system, and real-time prediction.


$$P(A|B) = \frac{P(B|A) · P(A)}{P(B)}$$


In the equation, $P$ stands for probability. $P(A)$ reads "the probability of Event A occurring" while $P(A|B)$ reads "the probability of Event A occurring *provided* Event B happened."  

Here is a practical Bayes Theorem application to solve any parent's problem:

> What is the chance that my kid will contract sickness if another kid she plays with has a sniffle?

Let `P(Sick)` stands for how often kids of the same age as that kid get sick in that given month, and `P(Sniffle)` for how often they have sniffles.

**Example**: 20% of kids age 3-5 in California Bay Area are sick and contagious during any given month of November (`P(Sick) = 0.2`), but even more kids with sniffles (30%) (`P(Sniffle) = 0.3`). It's been known that 80% of all sick kids have sniffles (`P(Sniff|Sick) = 0.8`).

```
P(Sick|Sniff) = (P(Sniff|Sick) * P(Sick)) / P(Sniff)
              = 0.8 * 0.2 / 0.3
              = 0.16 / 0.3
              = 0.53333...
```

Provided the kid has a sniffle, There is a **53%** chance that she will be sick.

So the formula kind of tells us “forwards” when we know “backwards” (or vice versa).

## Naive Bayes Classifier

Naive Bayes Classifier is basically a running application of Bayes Theorem.
It calculates the probability of every input...

In [1]:
# Activate current env
using Pkg
Pkg.activate(".")

"/Users/pieohpah/Code/julia/ham_spam/Project.toml"

In [2]:
using DataStructures
using Formatting

most_common(c::Accumulator) = most_common(c, length(c))
most_common(c::Accumulator, k) = sort(collect(c), by=kv->kv[2], rev=true)[1:k]

function isalpha(str)
    re = r"^[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)$"
    !occursin(re, str)
end

"Make a bag of words from email text."
function make_dictionary(root_dir)
    all_words = []
    emails = [joinpath(root_dir, f) for f in readdir(root_dir)]
    for (i, mail) in enumerate(emails)
        open(mail) do m
            for line in readlines(m)
                words = split(line)
                append!(all_words, words)
            end
        end
    end

    bag = counter(all_words)
    list_to_remove = [k for k in keys(bag)]

    for item in list_to_remove
        # remove if numerical
        if !isalpha(item)
            reset!(bag, item)
            # pop!(bag, item)
        elseif length(item) == 1
            reset!(bag, item)
            # pop!(bag, item)
        end
    end
    # Consider only most 3000 common words
    most_common(bag, 3000)
end

dict = make_dictionary("machine-learning-101/chapter1/train-mails")


┌ Info: Recompiling stale cache file /Users/pieohpah/.julia/compiled/v1.1/DataStructures/xKiwJ.ji for DataStructures [864edb3b-99cc-5e75-8d2d-829cb0a9cfe8]
└ @ Base loading.jl:1184


3000-element Array{Pair{Any,Int64},1}:
       "order" => 1414
          "'s" => 1356
     "address" => 1299
      "report" => 1217
        "mail" => 1133
    "language" => 1099
        "send" => 1080
       "email" => 1066
     "program" => 1009
         "our" => 991 
        "list" => 946 
         "one" => 921 
        "name" => 883 
               ⋮      
      "valley" => 8   
        "7-10" => 8   
        "ceth" => 8   
      "racism" => 8   
         "lar" => 8   
      "divide" => 8   
         "que" => 8   
 "punctuation" => 8   
     "transcr" => 8   
        "ongo" => 8   
         "vcs" => 8   
       "float" => 8   

In [3]:
# Playing with count
words = ["foo", "fum", "fast", "foo", "fum"]
for word in words
    c = count(x->x==word, words)
    println(c)
end

2
2
1
2
2


In [4]:
function A(mail_dir)
    files = [joinpath(mail_dir, f) for f in readdir(mail_dir)]
    features_matrix = zeros(length(files), 3000)
    train_labels = zeros(length(files))
    files
end


function B()
    files = A("machine-learning-101/chapter1/train-mails/")
    features_matrix = zeros(length(files), 3000)
    train_labels = zeros(length(files))
    count = 1
    doc_id = 1
    length(dict)
    for file in files
        open(file) do f
            for (i, line) in enumerate(readlines(f))
                if i == 3
                    words = split(line)
                    for word in words
                        word_id = 1

                        # Go through the bag of words
                        for (i, d) in enumerate(dict)
                            printfmt("d: {}", d[1])
                            if String(d[1]) == word
                                word_id = i
                                c = count(x -> x == word, words)
                                println(c)
                                features_matrix[doc_id, word_id] = c
                            end
                        end
                    end
                end
            end
        end
    end
    features_matrix
end

fm = B()
println(fm)


d: orderd: 'sd: addressd: reportd: maild: languaged: sendd: emaild: programd: ourd: listd: oned: named: received: freed: moneyd: workd: n'td: 3dd: Subject:d: informationd: businessd: pleased: universityd: e-maild: usd: dayd: followd: internetd: overd: calld: httpd: checkd: eachd: linguisticd: included: comd: needd: wantd: numberd: letterd: manyd: hered: marketd: startd: evend: faxd: formd: mostd: firstd: webd: serviced: interestd: softwared: readd: removed: thosed: weekd: creditd: everyd: sited: lld: englishd: edud: productd: muchd: bulkd: phoned: offerd: bestd: mustd: twod: costd: wwwd: computerd: linkd: stated: cardd: homed: ownd: red: ded: availabled: systemd: milliond: messaged: opportunityd: conferenced: afterd: questiond: placed: payd: withind: worldd: writed: helpd: hourd: selld: copyd: showd: belowd: samed: bookd: filed: throughd: directoryd: befored: easyd: wordd: whered: incomed: tryd: thankd: telld: todayd: companyd: memberd: exampled: advertised: backd: differentd: monthd: 

d: amd: purposed: sociolinguisticd: publisherd: dissertationd: seekd: austind: 3rdd: basicd: seriousd: negatived: mdd: demonstrated: ftpd: themed: mrd: mid: usaged: contrastd: hypothesisd: williamd: arabicd: contributiond: passwordd: newsgroupd: pmd: saturdayd: pasted: gruyterd: classifiedd: narratived: animald: representationd: wealthd: wrongd: indeedd: representd: mailboxd: guidelined: clearlyd: etd: pred: tipd: distributiond: dictionaryd: economyd: respectd: clothd: eventd: avoidd: properd: intendd: ild: forced: neard: falld: arrived: throughoutd: editd: machined: patternd: priord: announced: everydayd: boyd: duringd: financiallyd: servicesd: compatibled: combined: artd: overnightd: plenaryd: cabled: nound: affiliationd: owed: promoted: minimald: identityd: fontd: robertd: statementd: sponsord: martind: imaged: luckyd: stayd: labeld: sided: creoled: accommodationd: mucd: influenced: hrefd: goldd: throwd: occurd: commandd: basicallyd: albad: mexicod: relativelyd: coffeed: otherwised:

d: seand: childrend: cogscusd: bondd: combinationd: genred: specifyd: danield: smartd: junod: crimed: spotd: postcardd: spoused: reprintd: matthewd: n-aryd: allend: merelyd: remindd: psychologyd: mostlyd: crossd: administratived: amlapd: fairchildd: formationd: step-byd: ned: keyboardd: comfortd: birthd: disketted: svminusd: constructd: industriald: coind: dislocationd: nationd: skepticald: operatord: affectd: atlanticd: mathematicald: expertised: njd: logicald: surroundd: anywayd: illegald: classicald: eld: cted: refinanced: liddelld: motivated: removald: home-based: uscd: householdd: spokaned: facultyd: complaintd: picd: incorporated: uniond: msnd: totalsd: legallyd: rentald: airlined: mild: noned: pipelined: dud: amherstd: cramd: reciped: suitd: commerced: partnershipd: argued: tongued: pheromoned: fastestd: mbd: ratd: esqd: scoped: ever-growd: countyd: rewrited: slavicd: genuined: paidd: wonderfuld: busyd: uclad: logd: regiond: restrictd: preparedd: competed: ted: sericeld: celebra

MethodError: MethodError: objects of type Int64 are not callable

In [5]:
"Make features matrix and label vector"
function extract_features(mail_dir, dict)
    files = [joinpath(mail_dir, f) for f in readdir(mail_dir)]
    features_matrix = zeros(length(files), 3000)
    train_labels = zeros(length(files))
    # cnt = 1
    doc_id = 1
    for (n, file) in enumerate(files)
        open(file) do f
            for (i, line) in enumerate(readlines(f))
                # Skip the first subject line and second empty line
                if i == 3
                    words = split(line)
                    for word in words
                        word_id = 1
                        # Go through the bag of words
                        for (i, d) in enumerate(dict)
                            if d[1] == word
                                printfmt("Found {} in the bag\n", word)
                                word_id = i
                                c = count(x -> x == word, words)
                                features_matrix[doc_id, word_id] = c
                                printfmt("F[{}, {}] = {}\n", doc_id, word_id, c)
                            end
                        end
                    end
                end
            end

            train_labels[doc_id] = 0
            filepath_tokens = split(file, "/")
            last_token = filepath_tokens[length(filepath_tokens)]
            if startswith(last_token, "spmsg")
                train_labels[doc_id] = 1
                # TODO: ?
                # cnt += 1
            end
            doc_id += 1
        end
    end
    (features_matrix, train_labels)
end



(features, labels) = extract_features(
    "./machine-learning-101/chapter1/train-mails", dict
)


Found deat in the bag
F[1, 713] = 1
Found sun in the bag
F[1, 1831] = 1
Found dec in the bag
F[1, 1705] = 1
Found est in the bag
F[1, 1409] = 1
Found michael in the bag
F[1, 993] = 1
Found ca in the bag
F[1, 285] = 1
Found subject in the bag
F[1, 123] = 1
Found re in the bag
F[1, 81] = 1
Found query in the bag
F[1, 517] = 1
Found ask in the bag
F[1, 127] = 1
Found anything in the bag
F[1, 390] = 1
Found interest in the bag
F[1, 53] = 1
Found construction in the bag
F[1, 482] = 2
Found np in the bag
F[1, 1077] = 2
Found np in the bag
F[1, 1077] = 2
Found second in the bag
F[1, 267] = 1
Found much in the bag
F[1, 66] = 1
Found relate in the bag
F[1, 473] = 1
Found consider in the bag
F[1, 396] = 1
Found construction in the bag
F[1, 482] = 2
Found form in the bag
F[1, 48] = 1
Found discuss in the bag
F[1, 722] = 1
Found list in the bag
F[1, 11] = 1
Found late in the bag
F[1, 958] = 1
Found reduplication in the bag
F[1, 664] = 2
Found logical in the bag
F[1, 2237] = 1
Found sense in the ba

F[4, 1972] = 2
Found unfortunately in the bag
F[4, 1810] = 1
Found double in the bag
F[4, 1090] = 1
Found room in the bag
F[4, 561] = 3
Found available in the bag
F[4, 83] = 3
Found prefer in the bag
F[4, 920] = 1
Found stay in the bag
F[4, 1220] = 2
Found hotel in the bag
F[4, 709] = 2
Found recommend in the bag
F[4, 1433] = 1
Found hotel in the bag
F[4, 709] = 2
Found gent in the bag
F[4, 1501] = 13
Found tel in the bag
F[4, 770] = 3
Found minute in the bag
F[4, 319] = 1
Found walk in the bag
F[4, 1352] = 1
Found conference in the bag
F[4, 88] = 26
Found centre in the bag
F[4, 1052] = 3
Found contact in the bag
F[4, 128] = 3
Found gent in the bag
F[4, 1501] = 13
Found office in the bag
F[4, 271] = 1
Found gent in the bag
F[4, 1501] = 13
Found tel in the bag
F[4, 770] = 3
Found food in the bag
F[4, 1658] = 2
Found breakfast in the bag
F[4, 1972] = 2
Found serve in the bag
F[4, 1412] = 1
Found university in the bag
F[4, 24] = 6
Found eat in the bag
F[4, 1350] = 1
Found complex in the b

F[10, 2166] = 1
Found lake in the bag
F[10, 2780] = 1
Found city in the bag
F[10, 242] = 1
Found further in the bag
F[10, 279] = 1
Found information in the bag
F[10, 21] = 1
Found telephone in the bag
F[10, 451] = 1
Found email in the bag
F[10, 8] = 1
Found edu in the bag
F[10, 64] = 1
Found de in the bag
F[11, 82] = 14
Found en in the bag
F[11, 1335] = 6
Found le in the bag
F[11, 694] = 7
Found de in the bag
F[11, 82] = 14
Found linguistique in the bag
F[11, 1815] = 3
Found de in the bag
F[11, 82] = 14
Found de in the bag
F[11, 82] = 14
Found un in the bag
F[11, 2303] = 3
Found en in the bag
F[11, 1335] = 6
Found et in the bag
F[11, 1173] = 5
Found de in the bag
F[11, 82] = 14
Found cycle in the bag
F[11, 1960] = 1
Found et in the bag
F[11, 1173] = 5
Found la in the bag
F[11, 545] = 5
Found de in the bag
F[11, 82] = 14
Found la in the bag
F[11, 545] = 5
Found dan in the bag
F[11, 1533] = 3
Found le in the bag
F[11, 694] = 7
Found de in the bag
F[11, 82] = 14
Found la in the bag
F[11, 

F[18, 1064] = 1
Found anyone in the bag
F[18, 135] = 1
Found dr in the bag
F[18, 493] = 1
Found peter in the bag
F[18, 1050] = 1
Found phonetician in the bag
F[18, 2788] = 1
Found late in the bag
F[18, 958] = 1
Found edinburgh in the bag
F[18, 1010] = 1
Found elizabeth in the bag
F[18, 2352] = 1
Found college in the bag
F[18, 704] = 1
Found scotland in the bag
F[18, 1252] = 1
Found seek in the bag
F[18, 1138] = 1
Found one in the bag
F[18, 12] = 2
Found computer in the bag
F[18, 75] = 1
Found course in the bag
F[18, 213] = 1
Found french in the bag
F[18, 598] = 1
Found phonetic in the bag
F[18, 571] = 1
Found micro in the bag
F[18, 2683] = 1
Found move in the bag
F[18, 183] = 1
Found along in the bag
F[18, 566] = 1
Found need in the bag
F[18, 38] = 1
Found one in the bag
F[18, 12] = 2
Found ibm in the bag
F[18, 1039] = 1
Found help in the bag
F[18, 96] = 1
Found either in the bag
F[18, 350] = 1
Found search in the bag
F[18, 157] = 1
Found inquiry in the bag
F[19, 824] = 1
Found word in

F[25, 544] = 1
Found free in the bag
F[25, 15] = 1
Found local in the bag
F[25, 646] = 2
Found domain in the bag
F[25, 641] = 2
Found case in the bag
F[25, 172] = 1
Found although in the bag
F[25, 918] = 1
Found clear in the bag
F[25, 751] = 1
Found seem in the bag
F[25, 195] = 3
Found bind in the bag
F[25, 1356] = 3
Found local in the bag
F[25, 646] = 2
Found domain in the bag
F[25, 641] = 2
Found thought in the bag
F[25, 2459] = 1
Found example in the bag
F[25, 118] = 2
Found greatly in the bag
F[25, 1605] = 1
Found appreciate in the bag
F[25, 1295] = 1
Found interest in the bag
F[25, 53] = 2
Found post in the bag
F[25, 156] = 1
Found summary in the bag
F[25, 486] = 1
Found response in the bag
F[25, 125] = 1
Found receive in the bag
F[25, 14] = 1
Found thank in the bag
F[25, 113] = 1
Found steve in the bag
F[25, 1820] = 1
Found seegmiller in the bag
F[25, 2959] = 2
Found seegmiller in the bag
F[25, 2959] = 2
Found edu in the bag
F[25, 64] = 1
Found discussion in the bag
F[26, 323] = 

...

Found japanese in the bag
F[106, 606] = 1
Found native in the bag
F[106, 556] = 2
Found speaker in the bag
F[106, 238] = 2
Found english in the bag
F[106, 63] = 5
Found degree in the bag
F[106, 927] = 3
Found tesl in the bag
F[106, 2665] = 2
Found plan in the bag
F[106, 181] = 1
Found three in the bag
F[106, 356] = 3
Found english in the bag
F[106, 63] = 5
Found teacher in the bag
F[106, 365] = 1
Found preferably in the bag
F[106, 2028] = 1
Found native in the bag
F[106, 556] = 2
Found speaker in the bag
F[106, 238] = 2
Found english in the bag
F[106, 63] = 5
Found provide in the bag
F[106, 151] = 2
Found instructor in the bag
F[106, 1883] = 3
Found off in the bag
F[106, 148] = 1
Found campus in the bag
F[106, 1552] = 1
Found us in the bag
F[106, 26] = 2
Found payment in the bag
F[106, 351] = 3
Found us in the bag
F[106, 26] = 2
Found teach in the bag
F[106, 166] = 4
Found hour in the bag
F[106, 97] = 3
Found payment in the bag
F[106, 351] = 3
Found one in the bag
F[106, 12] = 3
Found 

Found fax in the bag
F[107, 47] = 1
Found email in the bag
F[107, 8] = 1
Found call in the bag
F[108, 31] = 1
Found participation in the bag
F[108, 980] = 1
Found first in the bag
F[108, 50] = 3
Found seminar in the bag
F[108, 889] = 4
Found computational in the bag
F[108, 630] = 3
Found linguistic in the bag
F[108, 35] = 5
Found computational in the bag
F[108, 630] = 3
Found linguistic in the bag
F[108, 35] = 5
Found department in the bag
F[108, 268] = 2
Found computer in the bag
F[108, 75] = 2
Found science in the bag
F[108, 399] = 3


Excessive output truncated after 524294 bytes.

Found february in the bag
F[108, 818] = 1
Found seminar in the bag
F[108, 889] = 4
Found intend in the bag
F[108, 1184] = 1
Found bring in the bag
F[108, 415] = 1
Found together in the bag
F[108, 594] = 1
Found researcher in the bag
F[108, 1671] = 1
Found area in the bag
F[108, 261] = 3
Found computational in the bag
F[108, 630] = 3
Found linguistic in the bag
F[108, 35] = 5
Found natural in the bag
F[108, 443] = 1
Found language in the bag
F[108, 6] = 1
Found process in the bag
F[108, 138] = 1
Found relate in the bag
F[108, 473] = 1
Found area in the bag
F[108, 261] = 3
Found formal in the bag
F[108, 719] = 2
Found linguistic in the bag
F[108, 35] = 5
Found theoretical in the bag
F[108, 881] = 1
Found linguistic in the bag
F[108, 35] = 5
Found cognitive in the bag
F[108, 697] = 1
Found science in the bag
F[108, 399] = 3
Found etc in the bag
F[108, 124] = 1
Found main in the bag
F[108, 610] = 1
Found objective in the bag
F[108, 1718] = 1
Found kind in the bag
F[108, 424] = 1
Found work

InterruptException: InterruptException:

In [25]:
using ScikitLearn
@sk_import naive_bayes: GaussianNB

TRAIN_DIR = "./machine-learning-101/chapter1/train-mails"
TEST_DIR = "./machine-learning-101/chapter1/test-mails"

(features_matrix, labels) = extract_features(TRAIN_DIR, dict)
(test_features_matrix, test_labels) = extract_features(TEST_DIR, dict)



Found deat in the bag
F[1, 713] = 1
Found sun in the bag
F[1, 1831] = 1
Found dec in the bag
F[1, 1705] = 1
Found est in the bag
F[1, 1409] = 1
Found michael in the bag
F[1, 993] = 1
Found ca in the bag
F[1, 285] = 1
Found subject in the bag
F[1, 123] = 1
Found re in the bag
F[1, 81] = 1
Found query in the bag
F[1, 517] = 1
Found ask in the bag
F[1, 127] = 1
Found anything in the bag
F[1, 390] = 1
Found interest in the bag
F[1, 53] = 1
Found construction in the bag
F[1, 482] = 2
Found np in the bag
F[1, 1077] = 2
Found np in the bag
F[1, 1077] = 2
Found second in the bag
F[1, 267] = 1
Found much in the bag
F[1, 66] = 1
Found relate in the bag
F[1, 473] = 1
Found consider in the bag
F[1, 396] = 1
Found construction in the bag
F[1, 482] = 2
Found form in the bag
F[1, 48] = 1
Found discuss in the bag
F[1, 722] = 1
Found list in the bag
F[1, 11] = 1
Found late in the bag
F[1, 958] = 1
Found reduplication in the bag
F[1, 664] = 2
Found logical in the bag
F[1, 2237] = 1
Found sense in the ba

Found favorite in the bag
F[2, 1806] = 1
Found lee in the bag
F[2, 1526] = 1
Found bitnet in the bag
F[2, 1127] = 1
Found department in the bag
F[2, 268] = 1
Found foreign in the bag
F[2, 576] = 1
Found language in the bag
F[2, 6] = 1
Found southern in the bag
F[2, 1347] = 1
Found illinoi in the bag
F[2, 1648] = 1
Found university in the bag
F[2, 24] = 1
Found il in the bag
F[2, 1185] = 1
Found 's in the bag
F[3, 2] = 2
Found much in the bag
F[3, 66] = 1
Found np in the bag
F[3, 1077] = 3
Found np in the bag
F[3, 1077] = 3
Found 's in the bag
F[3, 2] = 2
Found np in the bag
F[3, 1077] = 3
Found pro in the bag
F[3, 1104] = 1
Found quite in the bag
F[3, 587] = 1
Found listserv in the bag
F[4, 1958] = 1
Found international in the bag
F[4, 251] = 3
Found conference in the bag
F[4, 88] = 26
Found second in the bag
F[4, 267] = 1
Found circular in the bag
F[4, 619] = 2
Found february in the bag
F[4, 818] = 1
Found literature in the bag
F[4, 655] = 8
Found analysis in the bag
F[4, 232] = 4
Fou

F[4, 1972] = 2
Found unfortunately in the bag
F[4, 1810] = 1
Found double in the bag
F[4, 1090] = 1
Found room in the bag
F[4, 561] = 3
Found available in the bag
F[4, 83] = 3
Found prefer in the bag
F[4, 920] = 1
Found stay in the bag
F[4, 1220] = 2
Found hotel in the bag
F[4, 709] = 2
Found recommend in the bag
F[4, 1433] = 1
Found hotel in the bag
F[4, 709] = 2
Found gent in the bag
F[4, 1501] = 13
Found tel in the bag
F[4, 770] = 3
Found minute in the bag
F[4, 319] = 1
Found walk in the bag
F[4, 1352] = 1
Found conference in the bag
F[4, 88] = 26
Found centre in the bag
F[4, 1052] = 3
Found contact in the bag
F[4, 128] = 3
Found gent in the bag
F[4, 1501] = 13
Found office in the bag
F[4, 271] = 1
Found gent in the bag
F[4, 1501] = 13
Found tel in the bag
F[4, 770] = 3
Found food in the bag
F[4, 1658] = 2
Found breakfast in the bag
F[4, 1972] = 2
Found serve in the bag
F[4, 1412] = 1
Found university in the bag
F[4, 24] = 6
Found eat in the bag
F[4, 1350] = 1
Found complex in the b

F[10, 41] = 2
Found reference in the bag
F[10, 228] = 1
Found director in the bag
F[10, 1481] = 1
Found linguistic in the bag
F[10, 35] = 2
Found program in the bag
F[10, 9] = 2
Found build in the bag
F[10, 378] = 1
Found university in the bag
F[10, 24] = 2
Found salt in the bag
F[10, 2166] = 1
Found lake in the bag
F[10, 2780] = 1
Found city in the bag
F[10, 242] = 1
Found further in the bag
F[10, 279] = 1
Found information in the bag
F[10, 21] = 1
Found telephone in the bag
F[10, 451] = 1
Found email in the bag
F[10, 8] = 1
Found edu in the bag
F[10, 64] = 1
Found de in the bag
F[11, 82] = 14
Found en in the bag
F[11, 1335] = 6
Found le in the bag
F[11, 694] = 7
Found de in the bag
F[11, 82] = 14
Found linguistique in the bag
F[11, 1815] = 3
Found de in the bag
F[11, 82] = 14
Found de in the bag
F[11, 82] = 14
Found un in the bag
F[11, 2303] = 3
Found en in the bag
F[11, 1335] = 6
Found et in the bag
F[11, 1173] = 5
Found de in the bag
F[11, 82] = 14
Found cycle in the bag
F[11, 1960

F[17, 328] = 2
Found 's in the bag
F[17, 2] = 2
Found book in the bag
F[17, 103] = 2
Found mary in the bag
F[17, 667] = 1
Found borrow in the bag
F[17, 1293] = 2
Found bill in the bag
F[17, 269] = 1
Found 's in the bag
F[17, 2] = 2
Found want in the bag
F[17, 39] = 3
Found fix in the bag
F[17, 2168] = 1
Found car in the bag
F[17, 462] = 1
Found n't in the bag
F[17, 18] = 1
Found suggest in the bag
F[17, 761] = 1
Found check in the bag
F[17, 33] = 1
Found through in the bag
F[17, 105] = 1
Found modern in the bag
F[17, 741] = 1
Found icelandic in the bag
F[17, 2958] = 2
Found volume in the bag
F[17, 695] = 1
Found syntax in the bag
F[17, 398] = 1
Found semantic in the bag
F[17, 607] = 1
Found series in the bag
F[17, 1028] = 1
Found book in the bag
F[17, 103] = 2
Found please in the bag
F[17, 23] = 2
Found specific in the bag
F[17, 656] = 1
Found reference in the bag
F[17, 228] = 1
Found yet in the bag
F[17, 558] = 1
Found anything in the bag
F[17, 390] = 2
Found phenomenon in the bag
F[1

...

Found three in the bag
F[106, 356] = 3
Found english in the bag
F[106, 63] = 5
Found teacher in the bag
F[106, 365] = 1
Found preferably in the bag
F[106, 2028] = 1
Found native in the bag
F[106, 556] = 2
Found speaker in the bag
F[106, 238] = 2
Found english in the bag
F[106, 63] = 5
Found provide in the bag
F[106, 151] = 2
Found instructor in the bag
F[106, 1883] = 3
Found off in the bag
F[106, 148] = 1
Found campus in the bag
F[106, 1552] = 1
Found us in the bag
F[106, 26] = 2
Found payment in the bag
F[106, 351] = 3
Found us in the bag
F[106, 26] = 2
Found teach in the bag
F[106, 166] = 4
Found hour in the bag
F[106, 97] = 3
Found payment in the bag
F[106, 351] = 3
Found one in the bag
F[106, 12] = 3
Found additional in the bag
F[106, 346] = 2
Found hour in the bag
F[106, 97] = 3
Found teach in the bag
F[106, 166] = 4
Found health in the bag
F[106, 1474] = 1
Found insurance in the bag
F[106, 919] = 1
Found assistant in the bag
F[106, 1861] = 1
Found additional in the bag
F[106, 346

Found send in the bag
F[107, 7] = 1
Found complete in the bag
F[107, 250] = 1
Found enclose in the bag
F[107, 542] = 1
Found personal in the bag
F[107, 302] = 1
Found experience in the bag
F[107, 233] = 7
Found contact in the bag
F[107, 128] = 1
Found telephone in the bag
F[107, 451] = 1
Found number in the bag
F[107, 40] = 1
Found mr in the bag
F[107, 1148] = 1
Found jean in the bag
F[107, 1857] = 1
Found center in the bag
F[107, 435] = 3
Found manager in the bag
F[107, 2534] = 1
Found apple in the bag
F[107, 2588] = 3
Found research in the bag
F[107, 134] = 5
Found center in the bag
F[107, 435] = 3
Found institute in the bag
F[107, 582] = 2
Found system in the bag
F[107, 84] = 2
Found science in the bag
F[107, 399] = 2
Found singapore in the bag
F[107, 2309] = 4
Found tel in the bag
F[107, 770] = 1
Found fax in the bag
F[107, 47] = 1
Found email in the bag
F[107, 8] = 1
Found call in the bag
F[108, 31] = 1
Found participation in the bag
F[108, 980] = 1
Found first in the bag
F[108, 5

Excessive output truncated after 524289 bytes.

2652] = 1
Found international in the bag
F[111, 251] = 5
Found standard in the bag
F[111, 409] = 1
Found unless in the bag
F[111, 1296] = 1
Found one in the bag
F[111, 12] = 1
Found keep in the bag
F[111, 225] = 2
Found avoid in the bag
F[111, 1182] = 1
Found peter in the bag
F[111, 1050] = 1
Found computational in the bag
F[112, 630] = 2
Found ling in the bag
F[112, 1070] = 1
Found ed in the bag
F[112, 381] = 1
Found analysis in the bag
F[112, 232] = 1
Found machine in the bag
F[112, 1192] = 2
Found translation in the bag
F[112, 1016] = 2
Found de in the bag
F[112, 82] = 3


([1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 17.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [41]:
model = GaussianNB()
#train model
fit!(model, features_matrix, labels)
#predict
predicted_labels = predict(model, test_features_matrix)

@sk_import metrics: accuracy_score
accuracy = accuracy_score(test_labels, predicted_labels)

println(predicted_labels[132])

1.0


In [39]:
function openread(filename)
    open(filename) do f
        for line in readlines(f)
            println(line)
        end
    end
end

files = readdir(TEST_DIR)
for (i, file) in enumerate(files)
    if file == "myspam.txt"
        println(i)
        abs = joinpath(TEST_DIR, file)
        openread(abs)
    end
end

#=
last = joinpath(TEST_DIR, files[lastindex(files)])
println(last)
open(last) do f
    for line in readlines(f)
        println(line)
    end
end
=#

132
Dear my beloved,

I have a proposition. I'm sheik of ethiopia and I want to buy your asset.
